# Loading Data and Setup

Here, we load all the files related to the IPIA experiments and organize them into a file that helps govern the analysis of results.

In [2]:
import pandas as pd
import json
import glob
import os

In [3]:
def read_json_files_from_directory(directory_path):
    """
    Reads all JSON files from a specified directory and returns their content
    as a list of Python dictionaries.

    Args:
        directory_path (str): The path to the directory containing the JSON files.

    Returns:
        list: A list of dictionaries, where each dictionary represents the
              content of a JSON file.
    """
    json_data = []
    # Construct the pattern to find all .json files in the directory
    json_pattern = os.path.join(directory_path, '*.json')

    # Use glob to get a list of all matching file paths
    file_list = glob.glob(json_pattern)

    for file_path in file_list:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                json_data.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {file_path}: {e}")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An unexpected error occurred while reading {file_path}: {e}")
    return json_data

In [5]:
directory_path = "/home/dni138/mozilla_ai/data"

In [7]:
all_json_data = read_json_files_from_directory(directory_path)

In [13]:
json.loads(all_json_data[0]['dcarpintero_pangolin_guard_base_prompt_benign'][0])['valid']

True

In [15]:
list(all_json_data[0].keys())[0]

'dcarpintero_pangolin_guard_base_prompt_benign'

In [20]:
test_names = []
for dataset in all_json_data:
    test_names.append(list(dataset.keys())[0])

In [24]:
prefixes = []
for name in test_names:
    prefixes.append("_".join(name.split("_")[:-2]))

In [27]:
prefixes = list(set(prefixes))

In [28]:
prefixes

['ProtectAI_deberta_v3_base_prompt_injection_v2',
 'deepset_deberta_v3_base_injection',
 'ProtectAI_deberta_v3_small_prompt_injection_v2',
 'JasperLS_gelectra_base_injection',
 'leolee99_InjecGuard',
 'hbseong_HarmAug_Guard',
 'JasperLS_deberta_v3_base_injection',
 'dcarpintero_pangolin_guard_base',
 'ProtectAI_deberta_v3_base_prompt_injection',
 'qualifire_prompt_injection_sentinel']

In [32]:
from collections import defaultdict

prefix_to_name = defaultdict(list)
for prefix in prefixes:
    for name in test_names:
        if prefix == "_".join(name.split("_")[:-2]):
            prefix_to_name[prefix].append(name)

In [37]:
organized_data = {}
for prefix in prefixes:
    dict_to_frame = defaultdict(list)
    for test_name in prefix_to_name[prefix]:
        for dataset in all_json_data:
            if test_name in dataset.keys():
                column_name = test_name.split("_")[-1]
                for line in dataset[test_name]:
                    dict_to_frame[column_name].append(json.loads(line)['valid'])
    organized_data[prefix] = dict_to_frame

In [38]:
organized_data.keys()

dict_keys(['ProtectAI_deberta_v3_base_prompt_injection_v2', 'deepset_deberta_v3_base_injection', 'ProtectAI_deberta_v3_small_prompt_injection_v2', 'JasperLS_gelectra_base_injection', 'leolee99_InjecGuard', 'hbseong_HarmAug_Guard', 'JasperLS_deberta_v3_base_injection', 'dcarpintero_pangolin_guard_base', 'ProtectAI_deberta_v3_base_prompt_injection', 'qualifire_prompt_injection_sentinel'])

In [ ]:
with open("/home/dni138/mozilla_ai/data/organized_results.json", "w") as f:
    json.dump(organized_data, f)

# Analysis

Here, we analyze the results from the PIA experiment. Specifically, we look at the F1-Score, Precision, Recall, and Confusion Matrix.

In [3]:
with open("/home/dni138/mozilla_ai/data/organized_results.json", "r") as f:
    organized_data = json.load(f)

In [5]:
key = list(organized_data.keys())[0]

gt_email = [False] * len(organized_data[key]['email'])
gt_table = [False] * len(organized_data[key]['table'])
gt_benign = [True] * len(organized_data[key]['benign'])

In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

for key in organized_data.keys():
    dataset = organized_data[key]

    print("------EMAIL RESULTS-------")
    email_results = dataset['email'] + dataset['benign']
    gt_results = gt_email + gt_benign
    print(key)
    print("F1 Score: {}".format(f1_score(gt_results, email_results, pos_label=False)))
    print("Precision: {}".format(precision_score(gt_results, email_results, pos_label=False)))
    print("Recall: {}".format(recall_score(gt_results, email_results, pos_label=False)))
    print("Confusion Matrix: \n\n {} \n".format(confusion_matrix(gt_results, email_results, labels=[False, True])))

    print("------TABLE RESULTS-------")
    table_results = dataset['table'] + dataset['benign']
    gt_results = gt_table + gt_benign
    print(key)
    print("F1 Score: {}".format(f1_score(gt_results, table_results, pos_label=False)))
    print("Precision: {}".format(precision_score(gt_results, table_results, pos_label=False)))
    print("Recall: {}".format(recall_score(gt_results, table_results, pos_label=False)))
    print("Confusion Matrix: \n\n {} \n".format(confusion_matrix(gt_results, table_results, labels=[False, True])))

------EMAIL RESULTS-------
ProtectAI_deberta_v3_base_prompt_injection_v2
F1 Score: 0.6321148065548337
Precision: 0.6952858361774744
Recall: 0.5794666666666667
Confusion Matrix: 

 [[6519 4731]
 [2857 8391]] 

------TABLE RESULTS-------
ProtectAI_deberta_v3_base_prompt_injection_v2
F1 Score: 0.0741275206015266
Precision: 0.2546308374641273
Recall: 0.04337777777777778
Confusion Matrix: 

 [[  976 21524]
 [ 2857  8391]] 

------EMAIL RESULTS-------
deepset_deberta_v3_base_injection
F1 Score: 0.7805995004163198
Precision: 0.6401502219187436
Recall: 1.0
Confusion Matrix: 

 [[11250     0]
 [ 6324  4924]] 

------TABLE RESULTS-------
deepset_deberta_v3_base_injection
F1 Score: 0.8767827916764087
Precision: 0.7805995004163198
Recall: 1.0
Confusion Matrix: 

 [[22500     0]
 [ 6324  4924]] 

------EMAIL RESULTS-------
ProtectAI_deberta_v3_small_prompt_injection_v2
F1 Score: 0.2409760414691194
Precision: 0.41715911578025827
Recall: 0.16942222222222222
Confusion Matrix: 

 [[1906 9344]
 [2663 85